In [1]:
import tensorflow as tf
import numpy as np
import timeit

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import seaborn as sns

import os
os.environ["KMP_WARNINGS"] = "FALSE" 


import scipy
import scipy.io
import time

import sys

import scipy
import scipy.io
import time


import logging

import os.path
import shutil
from datetime import datetime
import pickle
class Sampler:
    # Initialize the class
    def __init__(self, dim, coords, func, name=None):
        self.dim = dim
        self.coords = coords
        self.func = func
        self.name = name

    def sample(self, N):
        x = self.coords[0:1, :] + (self.coords[1:2, :] - self.coords[0:1, :]) * np.random.rand(N, self.dim)
        y = self.func(x)
        return x, y
    

######################################################################################################
def u(x, a_1, a_2):
    return np.sin(a_1 * np.pi * x[:, 0:1]) * np.sin(a_2 * np.pi * x[:, 1:2])

def u_xx(x, a_1, a_2):
    return - (a_1 * np.pi) ** 2 * np.sin(a_1 * np.pi * x[:, 0:1]) * np.sin(a_2 * np.pi * x[:, 1:2])

def u_yy(x, a_1, a_2):
    return - (a_2 * np.pi) ** 2 * np.sin(a_1 * np.pi * x[:, 0:1]) * np.sin(a_2 * np.pi * x[:, 1:2])

# Forcing
def f(x, a_1, a_2, lam):
    return u_xx(x, a_1, a_2) + u_yy(x, a_1, a_2) + lam * u(x, a_1, a_2)

def operator(u, x1, x2, lam, sigma_x1=1.0, sigma_x2=1.0):
    u_x1 = tf.gradients(u, x1)[0] / sigma_x1
    u_x2 = tf.gradients(u, x2)[0] / sigma_x2
    u_xx1 = tf.gradients(u_x1, x1)[0] / sigma_x1
    u_xx2 = tf.gradients(u_x2, x2)[0] / sigma_x2
    residual = u_xx1 + u_xx2 + lam * u
    return residual
#######################################################################################################

class Helmholtz2D:
    def __init__(self, layers, operator, ics_sampler, bcs_sampler, res_sampler, lam, mode, sess):
        # Normalization constants


        self.mode = mode

        self.dirname, logpath = self.make_output_dir()
        self.logger = self.get_logger(logpath)     

        X, _ = res_sampler.sample(np.int32(1e5))
        self.mu_X, self.sigma_X = X.mean(0), X.std(0)
        self.mu_x1, self.sigma_x1 = self.mu_X[0], self.sigma_X[0]
        self.mu_x2, self.sigma_x2 = self.mu_X[1], self.sigma_X[1]

        # Samplers
        self.operator = operator
        self.ics_sampler = ics_sampler
        self.bcs_sampler = bcs_sampler
        self.res_sampler = res_sampler

        # Helmoholtz constant
        self.lam = tf.constant(lam, dtype=tf.float32)

        # Mode
        self.model = mode

        # Record stiff ratio
        # self.stiff_ratio = stiff_ratio

        # Adaptive constant
        self.rate = 0.5
        self.rho = 0.5
        self.T = 0.9

        self.adaptive_constant_bc_val = np.array(1.0)
        self.adaptive_constant_res_val = np.array(1.0)

        # Initialize network weights and biases
        self.layers = layers
        self.weights, self.biases = self.initialize_NN(layers)

        # if mode in ['M3', 'M4']:
        #     # Initialize encoder weights and biases
        #     self.encoder_weights_1 = self.xavier_init([2, layers[1]])
        #     self.encoder_biases_1 = self.xavier_init([1, layers[1]])

        #     self.encoder_weights_2 = self.xavier_init([2, layers[1]])
        #     self.encoder_biases_2 = self.xavier_init([1, layers[1]])

        # Define Tensorflow session
        self.sess = sess #tf.Session(config=tf.ConfigProto(log_device_placement=False))

        # Define placeholders and computational graph
        self.x1_u_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.x2_u_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.u_tf = tf.placeholder(tf.float32, shape=(None, 1))

        self.x1_bc1_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.x2_bc1_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.u_bc1_tf = tf.placeholder(tf.float32, shape=(None, 1))

        self.x1_bc2_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.x2_bc2_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.u_bc2_tf = tf.placeholder(tf.float32, shape=(None, 1))

        self.x1_bc3_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.x2_bc3_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.u_bc3_tf = tf.placeholder(tf.float32, shape=(None, 1))

        self.x1_bc4_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.x2_bc4_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.u_bc4_tf = tf.placeholder(tf.float32, shape=(None, 1))

        self.x1_r_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.x2_r_tf = tf.placeholder(tf.float32, shape=(None, 1))
        self.r_tf = tf.placeholder(tf.float32, shape=(None, 1))

        # Define placeholder for adaptive constant
        self.adaptive_constant_bc_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_bc_val.shape)
        self.adaptive_constant_res_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_res_val.shape)

        # Evaluate predictions
        self.u_bc1_pred = self.net_u(self.x1_bc1_tf, self.x2_bc1_tf)
        self.u_bc2_pred = self.net_u(self.x1_bc2_tf, self.x2_bc2_tf)
        self.u_bc3_pred = self.net_u(self.x1_bc3_tf, self.x2_bc3_tf)
        self.u_bc4_pred = self.net_u(self.x1_bc4_tf, self.x2_bc4_tf)

        self.u_pred = self.net_u(self.x1_u_tf, self.x2_u_tf)
        self.r_pred = self.net_r(self.x1_r_tf, self.x2_r_tf)

        # Boundary loss
        self.loss_bc1 = tf.reduce_mean(tf.square(self.u_bc1_tf - self.u_bc1_pred))
        self.loss_bc2 = tf.reduce_mean(tf.square(self.u_bc2_tf - self.u_bc2_pred))
        self.loss_bc3 = tf.reduce_mean(tf.square(self.u_bc3_tf - self.u_bc3_pred))
        self.loss_bc4 = tf.reduce_mean(tf.square(self.u_bc4_tf - self.u_bc4_pred))
        self.loss_bcs =  (self.loss_bc1 + self.loss_bc2 + self.loss_bc3 + self.loss_bc4)

        # Residual loss
        self.loss_res = tf.reduce_mean(tf.square(self.r_tf - self.r_pred))

        # Total loss
        self.loss = self.adaptive_constant_res_tf * self.loss_res + self.adaptive_constant_bc_tf * (self.loss_bc1 + self.loss_bc2 + self.loss_bc3 + self.loss_bc4)

        # Define optimizer with learning rate schedule
        self.global_step = tf.Variable(0, trainable=False)
        starter_learning_rate = 1e-3
        self.learning_rate = tf.train.exponential_decay(starter_learning_rate, self.global_step, 1000, 0.9, staircase=False)
        # Passing global_step to minimize() will increment it at each step.
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss, global_step=self.global_step)

        self.loss_tensor_list = [self.loss ,  self.loss_res ,  self.loss_bcs,  self.loss_bc1 , self.loss_bc2 , self.loss_bc3, self.loss_bc4] 
        self.loss_list = ["total loss" , "loss_res" , "loss_bcs" , "loss_bc1", "loss_bc2", "loss_bc3", "loss_bc4"] 



        self.prev_lam_bc =1
        self.prev_lam_res = 1
        self.index =1
        self.epoch_loss = dict.fromkeys(self.loss_list, 0)
        self.loss_history = dict((loss, []) for loss in self.loss_list)
        # Logger
        self.loss_bcs_log = []
        self.loss_res_log = []
        # self.saver = tf.train.Saver()

        # Generate dicts for gradients storage
        self.dict_gradients_res_layers = self.generate_grad_dict()
        self.dict_gradients_bcs_layers = self.generate_grad_dict()

        # Gradients Storage
        self.grad_res = []
        self.grad_bcs = []
        for i in range(len(self.layers) - 1):
            self.grad_res.append(tf.gradients(self.loss_res, self.weights[i])[0])
            self.grad_bcs.append(tf.gradients(self.loss_bcs, self.weights[i])[0])

        # Compute and store the adaptive constant
        self.adpative_constant_bc_log = []
        self.adpative_constant_res_log = []
        
        self.mean_grad_res_list = []
        self.mean_grad_bcs_list = []
        
        self.mean_grad_res_log = []
        self.mean_grad_bcs_log = []
    
        for i in range(len(self.layers) - 1):
            self.mean_grad_res_list.append(tf.reduce_mean(tf.abs(self.grad_res[i]))) 
            self.mean_grad_bcs_list.append(tf.reduce_mean(tf.abs(self.grad_bcs[i])))
        
        self.mean_grad_res = tf.reduce_mean(tf.stack(self.mean_grad_res_list))
        self.mean_grad_bcs = tf.reduce_mean(tf.stack(self.mean_grad_bcs_list))

        # # Stiff Ratio
        # if self.stiff_ratio:
        #     self.Hessian, self.Hessian_bcs, self.Hessian_res = self.get_H_op()
        #     self.eigenvalues, _ = tf.linalg.eigh(self.Hessian)
        #     self.eigenvalues_bcs, _ = tf.linalg.eigh(self.Hessian_bcs)
        #     self.eigenvalues_res, _ = tf.linalg.eigh(self.Hessian_res)

        #     self.eigenvalue_log = []
        #     self.eigenvalue_bcs_log = []
        #     self.eigenvalue_res_log = []

        # Initialize Tensorflow variables
        init = tf.global_variables_initializer()
        self.sess.run(init)

     # Create dictionary to store gradients
    def generate_grad_dict(self, layers):
        num = len(layers) - 1
        grad_dict = {}
        for i in range(num):
            grad_dict['layer_{}'.format(i + 1)] = []
        return grad_dict

    # Save gradients
    def save_gradients(self, tf_dict):
        num_layers = len(self.layers)
        for i in range(num_layers - 1):
            grad_res_value, grad_bcs_value = self.sess.run([self.grad_res[i], self.grad_bcs[i]], feed_dict=tf_dict)

            # save gradients of loss_res and loss_bcs
            self.dict_gradients_res_layers['layer_' + str(i + 1)].append(grad_res_value.flatten())
            self.dict_gradients_bcs_layers['layer_' + str(i + 1)].append(grad_bcs_value.flatten())
        return None



    # Xavier initialization
    def xavier_init(self,size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = 1. / np.sqrt((in_dim + out_dim) / 2.)
        return tf.Variable(tf.random_normal([in_dim, out_dim], dtype=tf.float32) * xavier_stddev,
                           dtype=tf.float32)

    # Initialize network weights and biases using Xavier initialization
    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        for l in range(0, num_layers - 1):
            W = self.xavier_init(size=[layers[l], layers[l + 1]])
            b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    # Evaluates the forward pass
    def forward_pass(self, H):
        num_layers = len(self.layers)
        for l in range(0, num_layers - 2):
            W = self.weights[l]
            b = self.biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = self.weights[-1]
        b = self.biases[-1]
        H = tf.add(tf.matmul(H, W), b)
        return H

    # Forward pass for u
    def net_u(self, x1, x2):
        u = self.forward_pass(tf.concat([x1, x2], 1))
        return u

    # Forward pass for residual
    def net_r(self, x1, x2):
        u = self.net_u(x1, x2)
        residual = self.operator(u, x1, x2,
                                 self.lam,
                                 self.sigma_x1,
                                 self.sigma_x2)
        return residual

    # Feed minibatch
    def fetch_minibatch(self, sampler, N):
        X, Y = sampler.sample(N)
        X = (X - self.mu_X) / self.sigma_X
        return X, Y


    def lambda_balance(self ,term , index):

        loss_list = ["loss_res" , "loss_bcs" ] 

        m = len(loss_list)
        num = np.exp(self.loss_history[term][-1] /(self.T * self.loss_history[term][index] +1e-12))
        denum = 0 

        for  key in loss_list:
            denum +=  np.exp(self.loss_history[key][-1] /(self.T * self.loss_history[key][index] +1e-12))
        return m * (num / denum)
     
    def relobralo( self, term ,prev_lamda ):

        lambda_hist = self.rho  * prev_lamda + (1.0 - self.rho) *  self.lambda_balance( term , 1)
        lambda_i = self.rate  * lambda_hist + (1.0 - self.rate) *  self.lambda_balance( term , self.index)

        return lambda_i
    
  # Trains the model by minimizing the MSE loss
    def trainmb(self, nIter=10000, batch_size=128):
        itValues = [1,100,1000,39999]

        start_time = timeit.default_timer()
        for it in range(1 , nIter):
            # Fetch boundary mini-batches
            X_bc1_batch, u_bc1_batch = self.fetch_minibatch(self.bcs_sampler[0], batch_size)
            X_bc2_batch, u_bc2_batch = self.fetch_minibatch(self.bcs_sampler[1], batch_size)
            X_bc3_batch, u_bc3_batch = self.fetch_minibatch(self.bcs_sampler[2], batch_size)
            X_bc4_batch, u_bc4_batch = self.fetch_minibatch(self.bcs_sampler[3], batch_size)

            # Fetch residual mini-batch
            X_res_batch, f_res_batch = self.fetch_minibatch(self.res_sampler, batch_size)

            # Define a dictionary for associating placeholders with data
            tf_dict = {self.x1_bc1_tf: X_bc1_batch[:, 0:1], self.x2_bc1_tf: X_bc1_batch[:, 1:2],
                       self.u_bc1_tf: u_bc1_batch,
                       self.x1_bc2_tf: X_bc2_batch[:, 0:1], self.x2_bc2_tf: X_bc2_batch[:, 1:2],
                       self.u_bc2_tf: u_bc2_batch,
                       self.x1_bc3_tf: X_bc3_batch[:, 0:1], self.x2_bc3_tf: X_bc3_batch[:, 1:2],
                       self.u_bc3_tf: u_bc3_batch,
                       self.x1_bc4_tf: X_bc4_batch[:, 0:1], self.x2_bc4_tf: X_bc4_batch[:, 1:2],
                       self.u_bc4_tf: u_bc4_batch,
                       self.x1_r_tf: X_res_batch[:, 0:1], self.x2_r_tf: X_res_batch[:, 1:2], self.r_tf: f_res_batch,
                       self.adaptive_constant_res_tf:  self.adaptive_constant_res_val,
                       self.adaptive_constant_bc_tf:  self.adaptive_constant_bc_val
                       }

            # Run the Tensorflow session to minimize the loss
            _ , batch_losses = self.sess.run( [  self.train_op , self.loss_tensor_list ] ,tf_dict)

            for loss_values, key in zip(batch_losses, self.loss_list):
                self.loss_history[key].append(loss_values)
 

            # Print
            if it % 100 == 0:
                elapsed = timeit.default_timer() - start_time
                loss_value ,  loss_bcs_value, loss_res_value = self.sess.run([self.loss, self.loss_bcs, self.loss_res] , tf_dict)

 
                self.print('It: %d, Loss: %.3e, Loss_bcs: %.3e, Loss_res: %.3e,Time: %.2f' % (it, loss_value, loss_bcs_value, loss_res_value, elapsed))

            if it % 100 == 0:

                self.adaptive_constant_res_val =  self.relobralo( "loss_res"  ,self.prev_lam_res )  #fres *  np.exp(self.rate * fres)  /norm

                self.adaptive_constant_bc_val =  self.relobralo( "loss_bcs"  ,self.prev_lam_bc )  # adaptive_constant_value * (1.0 - self.beta)+ self.beta * self.adaptive_constant_val

                self.print('adaptive_constant_res_val: %f' % (self.adaptive_constant_res_val))
                self.print('adaptive_constant_bc_val: %f' % (self.adaptive_constant_bc_val))

                self.adpative_constant_bc_log.append(self.adaptive_constant_bc_val)
                self.adpative_constant_res_log.append(self.adaptive_constant_res_val)

                mean_grad_bcs ,  max_grad_res = self.sess.run([self.mean_grad_bcs, self.mean_grad_res] , tf_dict)

                self.mean_grad_bcs_log.append(mean_grad_bcs)
                self.mean_grad_res_log.append(max_grad_res)

                self.prev_lam_bc = self.adaptive_constant_bc_val
                self.prev_lam_res = self.adaptive_constant_res_val
                self.index = it

            start_time = timeit.default_timer()

            if it in itValues:
                    self.plot_layerLoss(tf_dict , it)
                    self.print("Gradients information stored ...")

            sys.stdout.flush()
 


 # Trains the model by minimizing the MSE loss
    def train(self, bcbatch , fbatch , nIter=10000):
        itValues = [1,100,1000,39999]

        start_time = timeit.default_timer()
        # Fetch boundary mini-batches
        X_bc1_batch, u_bc1_batch = self.fetch_minibatch(self.bcs_sampler[0], bcbatch)
        X_bc2_batch, u_bc2_batch = self.fetch_minibatch(self.bcs_sampler[1], bcbatch)
        X_bc3_batch, u_bc3_batch = self.fetch_minibatch(self.bcs_sampler[2], bcbatch)
        X_bc4_batch, u_bc4_batch = self.fetch_minibatch(self.bcs_sampler[3], bcbatch)

        # Fetch residual mini-batch
        X_res_batch, f_res_batch = self.fetch_minibatch(self.res_sampler, fbatch)

        # Define a dictionary for associating placeholders with data
        tf_dict = {self.x1_bc1_tf: X_bc1_batch[:, 0:1], self.x2_bc1_tf: X_bc1_batch[:, 1:2],
                    self.u_bc1_tf: u_bc1_batch,
                    self.x1_bc2_tf: X_bc2_batch[:, 0:1], self.x2_bc2_tf: X_bc2_batch[:, 1:2],
                    self.u_bc2_tf: u_bc2_batch,
                    self.x1_bc3_tf: X_bc3_batch[:, 0:1], self.x2_bc3_tf: X_bc3_batch[:, 1:2],
                    self.u_bc3_tf: u_bc3_batch,
                    self.x1_bc4_tf: X_bc4_batch[:, 0:1], self.x2_bc4_tf: X_bc4_batch[:, 1:2],
                    self.u_bc4_tf: u_bc4_batch,
                    self.x1_r_tf: X_res_batch[:, 0:1], self.x2_r_tf: X_res_batch[:, 1:2], self.r_tf: f_res_batch,
                    self.adaptive_constant_res_tf:  self.adaptive_constant_res_val,
                    self.adaptive_constant_bc_tf:  self.adaptive_constant_bc_val
                    }

        for it in range(1 , nIter):

            # Run the Tensorflow session to minimize the loss
            _ , batch_losses = self.sess.run( [  self.train_op , self.loss_tensor_list ] ,tf_dict)

            for loss_values, key in zip(batch_losses, self.loss_list):
                self.loss_history[key].append(loss_values)
 

            # Print
            if it % 100 == 0:
                elapsed = timeit.default_timer() - start_time
                loss_value ,  loss_bcs_value, loss_res_value = self.sess.run([self.loss, self.loss_bcs, self.loss_res] , tf_dict)

 
                self.print('It: %d, Loss: %.3e, Loss_bcs: %.3e, Loss_res: %.3e,Time: %.2f' % (it, loss_value, loss_bcs_value, loss_res_value, elapsed))

            if it % 100 == 0:

                self.adaptive_constant_res_val =  self.relobralo( "loss_res"  ,self.prev_lam_res )  #fres *  np.exp(self.rate * fres)  /norm

                self.adaptive_constant_bc_val =  self.relobralo( "loss_bcs"  ,self.prev_lam_bc )  # adaptive_constant_value * (1.0 - self.beta)+ self.beta * self.adaptive_constant_val

                self.print('adaptive_constant_res_val: %f' % (self.adaptive_constant_res_val))
                self.print('adaptive_constant_bc_val: %f' % (self.adaptive_constant_bc_val))

                self.adpative_constant_bc_log.append(self.adaptive_constant_bc_val)
                self.adpative_constant_res_log.append(self.adaptive_constant_res_val)

                mean_grad_bcs ,  max_grad_res = self.sess.run([self.mean_grad_bcs, self.mean_grad_res] , tf_dict)

                self.mean_grad_bcs_log.append(mean_grad_bcs)
                self.mean_grad_res_log.append(max_grad_res)

                self.prev_lam_bc = self.adaptive_constant_bc_val
                self.prev_lam_res = self.adaptive_constant_res_val
                self.index = it

            start_time = timeit.default_timer()

            if it in itValues:
                    self.plot_layerLoss(tf_dict , it)
                    self.print("Gradients information stored ...")

            sys.stdout.flush()
 
    # Evaluates predictions at test points
    def predict_u(self, X_star):
        X_star = (X_star - self.mu_X) / self.sigma_X
        tf_dict = {self.x1_u_tf: X_star[:, 0:1], self.x2_u_tf: X_star[:, 1:2]}
        u_star = self.sess.run(self.u_pred, tf_dict)
        return u_star

    def predict_r(self, X_star):
        X_star = (X_star - self.mu_X) / self.sigma_X
        tf_dict = {self.x1_r_tf: X_star[:, 0:1], self.x2_r_tf: X_star[:, 1:2]}
        r_star = self.sess.run(self.r_pred, tf_dict)
        return r_star



  # ###############################################################################################################################################
   # 
   #  
    def plot_layerLoss(self , tf_dict , epoch):
        ## Gradients #
        num_layers = len(self.layers)
        for i in range(num_layers - 1):
            grad_res, grad_bc1    = self.sess.run([ self.grad_res[i],self.grad_bcs[i]], feed_dict=tf_dict)

            # save gradients of loss_r and loss_u
            self.dict_gradients_res_layers['layer_' + str(i + 1)].append(grad_res.flatten())
            self.dict_gradients_bcs_layers['layer_' + str(i + 1)].append(grad_bc1.flatten())

        num_hidden_layers = num_layers -1
        cnt = 1
        fig = plt.figure(4, figsize=(13, 4))
        for j in range(num_hidden_layers):
            ax = plt.subplot(1, num_hidden_layers, cnt)
            ax.set_title('Layer {}'.format(j + 1))
            ax.set_yscale('symlog')
            gradients_res = self.dict_gradients_res_layers['layer_' + str(j + 1)][-1]
            gradients_bc1 = self.dict_gradients_bcs_layers['layer_' + str(j + 1)][-1]

            sns.distplot(gradients_res, hist=False,kde_kws={"shade": False},norm_hist=True,  label=r'$\nabla_\theta \mathcal{L}_r$')

            sns.distplot(gradients_bc1, hist=False,kde_kws={"shade": False},norm_hist=True,   label=r'$\nabla_\theta \mathcal{L}_{u_{bc1}}$')

            #ax.get_legend().remove()
            ax.set_xlim([-1.0, 1.0])
            #ax.set_ylim([0, 150])
            cnt += 1
        handles, labels = ax.get_legend_handles_labels()

        fig.legend(handles, labels, loc="center",  bbox_to_anchor=(0.5, -0.03),borderaxespad=0,bbox_transform=fig.transFigure, ncol=2)
        text = 'layerLoss_epoch' + str(epoch) +'.png'
        plt.savefig(os.path.join(self.dirname,text) , bbox_inches='tight')
        plt.close("all")
    # #########################
    # def make_output_dir(self):
        
    #     if not os.path.exists("/okyanus/users/afarea/PINN/Adaptive_PINN/IB_PINN/checkpoints"):
    #         os.mkdir("/okyanus/users/afarea/PINN/Adaptive_PINN/IB_PINN/checkpoints")
    #     dirname = os.path.join("/okyanus/users/afarea/PINN/Adaptive_PINN/IB_PINN/checkpoints", datetime.now().strftime("%b-%d-%Y_%H-%M-%S-%f_") + self.mode)
    #     os.mkdir(dirname)
    #     text = 'output.log'
    #     logpath = os.path.join(dirname, text)
    #     shutil.copyfile('/okyanus/users/afarea/PINN/Adaptive_PINN/IB_PINN/M2.py', os.path.join(dirname, 'M2.py'))

    #     return dirname, logpath
    
    # # ###########################################################
    def make_output_dir(self):
        
        if not os.path.exists("checkpoints"):
            os.mkdir("checkpoints")
        dirname = os.path.join("checkpoints", datetime.now().strftime("%b-%d-%Y_%H-%M-%S-%f_") + self.mode)
        os.mkdir(dirname)
        text = 'output.log'
        logpath = os.path.join(dirname, text)
        shutil.copyfile('M2.ipynb', os.path.join(dirname, 'M2.ipynb'))
        return dirname, logpath
    

    def get_logger(self, logpath):
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.DEBUG)
        sh = logging.StreamHandler()
        sh.setLevel(logging.DEBUG)        
        sh.setFormatter(logging.Formatter('%(message)s'))
        fh = logging.FileHandler(logpath)
        logger.addHandler(sh)
        logger.addHandler(fh)
        return logger


   
    def print(self, *args):
        for word in args:
            if len(args) == 1:
                self.logger.info(word)
            elif word != args[-1]:
                for handler in self.logger.handlers:
                    handler.terminator = ""
                if type(word) == float or type(word) == np.float64 or type(word) == np.float32: 
                    self.logger.info("%.4e" % (word))
                else:
                    self.logger.info(word)
            else:
                for handler in self.logger.handlers:
                    handler.terminator = "\n"
                if type(word) == float or type(word) == np.float64 or type(word) == np.float32:
                    self.logger.info("%.4e" % (word))
                else:
                    self.logger.info(word)


    def plot_loss_history(self , path):

        fig, ax = plt.subplots()
        fig.set_size_inches([15,8])
        for key in self.loss_history:
            self.print("Final loss %s: %e" % (key, self.loss_history[key][-1]))
            ax.semilogy(self.loss_history[key], label=key)
        ax.set_xlabel("epochs", fontsize=15)
        ax.set_ylabel("loss", fontsize=15)
        ax.tick_params(labelsize=15)
        ax.legend()
        plt.savefig(path)
        plt.close("all" , )
       #######################
    def save_NN(self):

        uv_weights = self.sess.run(self.weights)
        uv_biases = self.sess.run(self.biases)

        with open(os.path.join(self.dirname,'model.pickle'), 'wb') as f:
            pickle.dump([uv_weights, uv_biases], f)
            self.print("Save uv NN parameters successfully in %s ..." , self.dirname)

        # with open(os.path.join(self.dirname,'loss_history_BFS.pickle'), 'wb') as f:
        #     pickle.dump(self.loss_rec, f)
        with open(os.path.join(self.dirname,'loss_history_BFS.png'), 'wb') as f:
            self.plot_loss_history(f)


    def assign_batch_losses(self, batch_losses):
        for loss_values, key in zip(batch_losses, self.epoch_loss):
            self.epoch_loss[key] = loss_values


    def generate_grad_dict(self):
        num = len(self.layers) - 1
        grad_dict = {}
        for i in range(num):
            grad_dict['layer_{}'.format(i + 1)] = []
        return grad_dict
    
    def assign_batch_losses(self, batch_losses):
        for loss_values, key in zip(batch_losses, self.epoch_loss):
            self.epoch_loss[key] = loss_values
            
    def plt_prediction(self , x1 , x2 , X_star , u_star , u_pred , f_star , f_pred):
        from mpl_toolkits.axes_grid1 import ImageGrid

        ### Plot ###

        # Exact solution & Predicted solution
        # Exact soluton
        U_star = griddata(X_star, u_star.flatten(), (x1, x2), method='cubic')
        F_star = griddata(X_star, f_star.flatten(), (x1, x2), method='cubic')

        # Predicted solution
        U_pred = griddata(X_star, u_pred.flatten(), (x1, x2), method='cubic')
        F_pred = griddata(X_star, f_pred.flatten(), (x1, x2), method='cubic')

        titles = ['Exact $u(x)$' , 'Predicted $u(x)$' , 'Absolute error' , 'Exact $f(x)$' , 'Predicted $f(x)$' , 'Absolute error']
        data = [U_star , U_pred ,  np.abs(U_star - U_pred) , F_star , F_pred ,  np.abs(F_star - F_pred) ]
        

        fig_1 = plt.figure(1, figsize=(13, 5))
        grid = ImageGrid(fig_1, 111, direction="row", nrows_ncols=(2,3), 
                        label_mode="1", axes_pad=1.7, share_all=False, 
                        cbar_mode="each", cbar_location="right", 
                        cbar_size="5%", cbar_pad=0.0)
    # CREATE ARGUMENTS DICT FOR CONTOURPLOTS
        minmax_list = []
        kwargs_list = []
        for d in data:
            # if(local):
            #     minmax_list.append([np.min(d), np.max(d)])
            # else:
            minmax_list.append([np.min(d), np.max(d)])

            kwargs_list.append(dict(levels=np.linspace(minmax_list[-1][0],minmax_list[-1][1], 60),
                cmap="coolwarm", vmin=minmax_list[-1][0], vmax=minmax_list[-1][1]))

        for ax, z, kwargs, minmax, title in zip(grid, data, kwargs_list, minmax_list, titles):
        #pcf = [ax.tricontourf(x, y, z[0,:], **kwargs)]
            #pcfsets.append(pcf)
            # if (timeStp == 0):
                #  print( z[timeStp,:,:])
            pcf = [ax.pcolor(x1, x2, z , cmap='jet')]
            cb = ax.cax.colorbar(pcf[0], ticks=np.linspace(minmax[0],minmax[1],7),  format='%.3e')
            ax.cax.tick_params(labelsize=14.5)
            ax.set_title(title, fontsize=14.5, pad=7)
            ax.set_ylabel("y", labelpad=14.5, fontsize=14.5, rotation="horizontal")
            ax.set_xlabel("x", fontsize=14.5)
            ax.tick_params(labelsize=14.5)
            ax.set_xlim(x1.min(), x1.max())
            ax.set_ylim(x2.min(), x2.max())
            ax.set_aspect("equal")

        fig_1.set_size_inches(15, 10, True)
        fig_1.subplots_adjust(left=0.7, bottom=0, right=2.2, top=0.5, wspace=None, hspace=None)
        plt.tight_layout()
        plt.savefig(os.path.join(self.dirname,"prediction.png"), dpi=300 , bbox_inches='tight')
        plt.close("all" , )


    def plot_lambda(self ):

        fig, ax = plt.subplots()
        fig.set_size_inches([15,8])
        ax.semilogy(self.adpative_constant_bc_log, label=r'$\lambda_{bc}$')
        ax.semilogy(self.adpative_constant_res_log, label=r'$\lambda_{res}$')
        ax.set_xlabel("epochs", fontsize=15)
        ax.set_ylabel(r'$\lambda$', fontsize=15)
        ax.tick_params(labelsize=15)
        ax.legend()
        path = os.path.join(self.dirname,'lambda_history.png')
        plt.savefig(path)
        plt.close("all" , )





    def plot_grad(self ):

        fig, ax = plt.subplots()
        fig.set_size_inches([15,8])
        ax.semilogy(self.mean_grad_bcs_log, label=r'$\bar{\nabla_\theta \mathcal{L}_{u_{bc}}}$')
        ax.semilogy(self.mean_grad_res_log, label=r'$\bar{\nabla_\theta \mathcal{L}_{u_{res}}}$')
        ax.set_xlabel("epochs", fontsize=15)
        ax.set_ylabel(r'$\bar{\nabla_{\theta} \mathcal{L}}$', fontsize=15)
        ax.tick_params(labelsize=15)
        ax.legend()
        path = os.path.join(self.dirname,'grad_history.png')
        plt.savefig(path)
        plt.close("all" , )


In [2]:
#test_method(mtd , layers, operator, ics_sampler, bcs_sampler , res_sampler ,lam , mode , stiff_ratio , X_star ,u_star , f_star , nIter ,mbbatch_size , bcbatch_size , ubatch_size)

def test_method(method , layers, operator, ics_sampler, bcs_sampler, res_sampler, lam ,mode , stiff_ratio ,  X_star , u_star , f_star , nIter ,mbbatch_size , bcbatch_size , ubatch_size):


    model = Helmholtz2D(layers, operator, ics_sampler, bcs_sampler, res_sampler, lam, mode, stiff_ratio)

    # Train model
    start_time = time.time()

    if method =="full_batch":
        model.train(nIter  ,bcbatch_size , ubatch_size )
    elif method =="mini_batch":
        model.trainmb(nIter, batch_size=mbbatch_size)
    else:
        print("unknown method!")
    elapsed = time.time() - start_time


    # Predictions
    u_pred = model.predict_u(X_star)
    f_pred = model.predict_r(X_star)

    # Relative error
    error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    error_f = np.linalg.norm(f_star - f_pred, 2) / np.linalg.norm(f_star, 2)

    print('Relative L2 error_u: {:.2e}'.format(error_u))
    print('Relative L2 error_f: {:.2e}'.format(error_f))

    return [elapsed, error_u , error_f ,  model]



In [3]:

################################################################################################


nIter =40001
bcbatch_size = 500
ubatch_size = 5000
mbbatch_size = 128

a_1 = 1
a_2 = 4

# Parameter
lam = 1.0

# Domain boundaries
bc1_coords = np.array([[-1.0, -1.0], [1.0, -1.0]])
bc2_coords = np.array([[1.0, -1.0], [1.0, 1.0]])
bc3_coords = np.array([[1.0, 1.0], [-1.0, 1.0]])
bc4_coords = np.array([[-1.0, 1.0], [-1.0, -1.0]])

dom_coords = np.array([[-1.0, -1.0], [1.0, 1.0]])


# Train model

# Test data
nn = 100
x1 = np.linspace(dom_coords[0, 0], dom_coords[1, 0], nn)[:, None]
x2 = np.linspace(dom_coords[0, 1], dom_coords[1, 1], nn)[:, None]
x1, x2 = np.meshgrid(x1, x2)
X_star = np.hstack((x1.flatten()[:, None], x2.flatten()[:, None]))




# Exact solution
u_star = u(X_star, a_1, a_2)
f_star = f(X_star, a_1, a_2, lam)

# Create initial conditions samplers
ics_sampler = None

# Define model
mode = 'relobralo'            # Method: 'M1', 'M2', 'M3', 'M4'
stiff_ratio = False    # Log the eigenvalues of Hessian of losses

layers = [2, 50, 50, 50, 1]


iterations = 1
methods = ["full_batch"]

result_dict =  dict((mtd, []) for mtd in methods)

for mtd in methods:
    print("Method: ", mtd)
    time_list = []
    error_u_list = []
    error_f_list = []
    
    for index in range(iterations):

        print("Epoch: ", str(index+1))


        # Create boundary conditions samplers
        bc1 = Sampler(2, bc1_coords, lambda x: u(x, a_1, a_2), name='Dirichlet BC1')
        bc2 = Sampler(2, bc2_coords, lambda x: u(x, a_1, a_2), name='Dirichlet BC2')
        bc3 = Sampler(2, bc3_coords, lambda x: u(x, a_1, a_2), name='Dirichlet BC3')
        bc4 = Sampler(2, bc4_coords, lambda x: u(x, a_1, a_2), name='Dirichlet BC4')
        bcs_sampler = [bc1, bc2, bc3, bc4]

        # Create residual sampler
        res_sampler = Sampler(2, dom_coords, lambda x: f(x, a_1, a_2, lam), name='Forcing')

        # [elapsed, error_u , error_f ,  mode] = test_method(mtd , layers, operator, ics_sampler, bcs_sampler , res_sampler ,lam , mode , 
        #                                                                stiff_ratio , X_star ,u_star , f_star , nIter ,bcbatch_size , bcbatch_size , ubatch_size)

#test_method(mtd , layers, operator, ics_sampler, bcs_sampler , res_sampler ,lam , mode , stiff_ratio , X_star ,u_star , f_star , nIter ,mbbatch_size , bcbatch_size , ubatch_size)


        tf.reset_default_graph()
        gpu_options = tf.GPUOptions(visible_device_list="0")
        with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=False, log_device_placement=False)) as sess:
            model = Helmholtz2D(layers, operator, ics_sampler, bcs_sampler, res_sampler, lam, mode, sess)
 #def __init__(self, layers, operator, ics_sampler, bcs_sampler, res_sampler, lam, mode, sess)
            # Train model
            start_time = time.time()

            if mtd =="full_batch":
                model.train(  bcbatch_size , ubatch_size , nIter )
            elif mtd =="mini_batch":
                model.trainmb( mbbatch_size  , nIter)
            else:
                model.print("unknown method!")
            elapsed = time.time() - start_time


            # Predictions
            u_pred = model.predict_u(X_star)
            f_pred = model.predict_r(X_star)

            # Relative error
            error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
            error_f = np.linalg.norm(f_star - f_pred, 2) / np.linalg.norm(f_star, 2)

            model.print('Relative L2 error_u: {:.2e}'.format(error_u))
            model.print('Relative L2 error_f: {:.2e}'.format(error_f))

            model.plot_lambda()
            model.plot_grad()
            model.save_NN()
            model.plt_prediction( x1 , x2 , X_star , u_star , u_pred , f_star , f_pred)

            model.print("average adpative_constant_bc_log : " , np.average(model.adpative_constant_bc_log))
            model.print("average adpative_constant_res_log : " ,  np.average(model.adpative_constant_res_log))
            # sess.close()  

            time_list.append(elapsed)
            error_u_list.append(error_u)
            error_f_list.append(error_f)

    model.print("\n\nMethod: ", mtd)
    model.print("\naverage of time_list:" , sum(time_list) / len(time_list) )
    model.print("average of error_u_list:" , sum(error_u_list) / len(error_u_list) )
    model.print("average of error_v_list:" , sum(error_f_list) / len(error_f_list) )

    result_dict[mtd] = [time_list ,error_u_list ,error_f_list ]
    # scipy.io.savemat("M2_result_"+str(iterations)+"_"+mtd+".mat" , {'time_list':np.array(time_list),'error_u_list':np.array(error_u_list),'error_f_list':np.array(error_f_list)})

    scipy.io.savemat(os.path.join(model.dirname,""+mtd+"_Helmholtz_"+mode+"_result_mb"+str(mbbatch_size)+"_fb"+str(ubatch_size)+"_bc"+str(mbbatch_size)+"_exp"+str(bcbatch_size)+"nIter"+str(nIter)+".mat") , result_dict)




Method:  full_batch
Epoch:  1






2023-12-17 23:29:45.204495: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-17 23:29:45.235358: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2899885000 Hz
2023-12-17 23:29:45.235833: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56343fd008c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-12-17 23:29:45.235845: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-12-17 23:29:45.237464: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


/home/afrah2/anaconda3/envs/twoPhase/lib/python3.7/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/afrah2/anaconda3/envs/twoPhase/lib/python3.7/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/afrah2/anaconda3/envs/twoPhase/lib/python3.7/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `di

In [ ]:

Method: mini_batch

average of time_list:2.7972e+02
average of error_u_list:1.4240e-01
average of error_v_list:7.3166e-03

SyntaxError: invalid syntax (1497590327.py, line 3)

In [ ]:
model.loss_history

{'total loss': [0,
  6837.2354,
  7007.1406,
  7454.5513,
  6319.552,
  6783.9673,
  7334.4707,
  7050.1016,
  6187.1094,
  7036.614,
  5874.5605,
  6618.368,
  6937.3755,
  7219.1294,
  7099.772,
  6125.392,
  6735.928,
  6401.3887,
  7092.786,
  7017.5723,
  7781.6895,
  7438.0103,
  7742.739,
  7142.6064,
  7664.891,
  6878.4653,
  7327.2812,
  7257.5376,
  6311.009,
  6721.8486,
  7712.5347,
  7680.7427,
  7072.624,
  6979.8364,
  5573.472,
  6322.7134,
  6176.224,
  7239.36,
  7936.7607,
  8020.731,
  5830.1816,
  6709.6885,
  8016.3086,
  6610.1006,
  6139.4272,
  7068.506,
  7298.9165,
  7069.425,
  7903.4688,
  7141.165,
  6307.4526,
  6504.12,
  6209.652,
  6289.9307,
  6624.553,
  5486.6113,
  6898.023,
  6214.5635,
  5725.3936,
  6499.5776,
  6627.588,
  7278.8633,
  7669.574,
  5512.935,
  8063.378,
  6745.589,
  7422.527,
  8541.348,
  6321.757,
  6969.367,
  6339.0366,
  7387.566,
  7699.12,
  6587.506,
  6313.7646,
  7086.5728,
  6412.5703,
  6962.434,
  6666.712,
  5498

In [4]:
### Plot ###

# Exact solution & Predicted solution
# Exact soluton
U_star = griddata(X_star, u_star.flatten(), (x1, x2), method='cubic')
F_star = griddata(X_star, f_star.flatten(), (x1, x2), method='cubic')

# Predicted solution
U_pred = griddata(X_star, u_pred.flatten(), (x1, x2), method='cubic')
F_pred = griddata(X_star, f_pred.flatten(), (x1, x2), method='cubic')

fig_1 = plt.figure(1, figsize=(18, 5))
plt.subplot(1, 3, 1)
plt.pcolor(x1, x2, U_star, cmap='jet')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.title('Exact $u(x)$')

plt.subplot(1, 3, 2)
plt.pcolor(x1, x2, U_pred, cmap='jet')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.title('Predicted $u(x)$')

plt.subplot(1, 3, 3)
plt.pcolor(x1, x2, np.abs(U_star - U_pred), cmap='jet')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.title('Absolute error')
plt.tight_layout()
plt.show()

# Residual loss & Boundary loss
loss_res = mode.loss_res_log
loss_bcs = mode.loss_bcs_log

fig_2 = plt.figure(2)
ax = fig_2.add_subplot(1, 1, 1)
ax.plot(loss_res, label='$\mathcal{L}_{r}$')
ax.plot(loss_bcs, label='$\mathcal{L}_{u_b}$')
ax.set_yscale('log')
ax.set_xlabel('iterations')
ax.set_ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

# Adaptive Constant
adaptive_constant = mode.adpative_constant_log

fig_3 = plt.figure(3)
ax = fig_3.add_subplot(1, 1, 1)
ax.plot(adaptive_constant, label='$\lambda_{u_b}$')
ax.set_xlabel('iterations')
plt.legend()
plt.tight_layout()
plt.show()

# Gradients at the end of training
data_gradients_res = mode.dict_gradients_res_layers
data_gradients_bcs = mode.dict_gradients_bcs_layers

gradients_res_list = []
gradients_bcs_list = []

num_hidden_layers = len(layers) - 1
for j in range(num_hidden_layers):
    gradient_res = data_gradients_res['layer_' + str(j + 1)][-1]
    gradient_bcs = data_gradients_bcs['layer_' + str(j + 1)][-1]

    gradients_res_list.append(gradient_res)
    gradients_bcs_list.append(gradient_bcs)

cnt = 1
fig_4 = plt.figure(4, figsize=(13, 4))
for j in range(num_hidden_layers):
    ax = plt.subplot(1, 4, cnt)
    ax.set_title('Layer {}'.format(j + 1))
    ax.set_yscale('symlog')
    gradients_res = data_gradients_res['layer_' + str(j + 1)][-1]
    gradients_bcs = data_gradients_bcs['layer_' + str(j + 1)][-1]
    sns.distplot(gradients_bcs, hist=False,
                    kde_kws={"shade": False},
                    norm_hist=True, label=r'$\nabla_\theta \lambda_{u_b} \mathcal{L}_{u_b}$')
    sns.distplot(gradients_res, hist=False,
                    kde_kws={"shade": False},
                    norm_hist=True, label=r'$\nabla_\theta \mathcal{L}_r$')
    
    ax.get_legend().remove()
    ax.set_xlim([-3.0, 3.0])
    ax.set_ylim([0,100])
    cnt += 1
handles, labels = ax.get_legend_handles_labels()
fig_4.legend(handles, labels, loc="upper left", bbox_to_anchor=(0.35, -0.01),
            borderaxespad=0, bbox_transform=fig_4.transFigure, ncol=2)
plt.tight_layout()
plt.show()

# Eigenvalues if applicable
if stiff_ratio:
    eigenvalues_list = mode.eigenvalue_log
    eigenvalues_bcs_list = mode.eigenvalue_bcs_log
    eigenvalues_res_list = mode.eigenvalue_res_log
    eigenvalues_res = eigenvalues_res_list[-1]
    eigenvalues_bcs = eigenvalues_bcs_list[-1]

    fig_5 = plt.figure(5)
    ax = fig_5.add_subplot(1, 1, 1)
    ax.plot(eigenvalues_res, label='$\mathcal{L}_r$')
    ax.plot(eigenvalues_bcs, label='$\mathcal{L}_{u_b}$')
    ax.set_xlabel('index')
    ax.set_ylabel('eigenvalue')
    ax.set_yscale('symlog')
    plt.legend()
    plt.tight_layout()
    plt.show()

    

NameError: name 'u_pred' is not defined